In [2]:
import os
import cv2
from math import *
import numpy as np

In [3]:
def read_matrix_from_txt(file_path):
    with open(file_path, 'r') as f:
        matrix = np.array([list(map(float, line.split())) for line in f])
        return matrix.reshape((4, 4))

### methods
- CALIB_HAND_EYE_TSAI
- CALIB_HAND_EYE_PARK
- CALIB_HAND_EYE_HORAUD
- CALIB_HAND_EYE_ANDREFF
- CALIB_HAND_EYE_DANIILIDIS


In [114]:
R_gripper2base_list = []
T_gripper2base_list = []

R_target2camera_list = []
T_target2camera_list = []

input_dir_1 = '/Users/coulson/23.7.9/calibration/complete_cali'
input_dir_2 = '/Users/coulson/23.7.9/calibration/complete_real'

if os.path.exists(input_dir_1 + '/.DS_Store'):
    os.remove(input_dir_1 + '/.DS_Store')

if os.path.exists(input_dir_2 + '/.DS_Store'):
    os.remove(input_dir_2 + '/.DS_Store')
    
files_1 = sorted(os.listdir(input_dir_1))
for i in range(0, len(files_1)):
    mat = read_matrix_from_txt(os.path.join(input_dir_1, files_1[i]))
    R_target2camera_list.append(mat[:3, :3]) 
    T_target2camera_list.append(mat[:3, 3].reshape((3, 1)))       
    

files_2 = sorted(os.listdir(input_dir_2))
for i in range(0, len(files_2)):
    mat = read_matrix_from_txt(os.path.join(input_dir_2, files_2[i]))
    R_gripper2base_list.append(mat[:3, :3]) 
    T_gripper2base_list.append(mat[:3, 3].reshape((3, 1)))   
    
R_camera2gripper, T_camera2gripper = cv2.calibrateHandEye(R_gripper2base_list, T_gripper2base_list,
                                                            R_target2camera_list, T_target2camera_list, method = cv2.CALIB_HAND_EYE_DANIILIDIS)
RT_camera_to_gripper = np.column_stack((R_camera2gripper, T_camera2gripper))
RT_camera_to_gripper = np.row_stack((RT_camera_to_gripper, np.array([0, 0, 0, 1])))
assert(np.isclose(np.linalg.det(RT_camera_to_gripper[:3, :3]), 1.))
RT_camera_to_gripper # camera 2 rigid body

array([[ 0.99943167,  0.01557992,  0.02989315, -0.04248998],
       [-0.03179406,  0.14099791,  0.98949923,  0.02602853],
       [ 0.01120145, -0.98988729,  0.14141312, -0.01045452],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [43]:
def check_result(R_cb, T_cb, R_gb, T_gb, R_tc, T_tc):
        for i in range(len(R_gb)):
            RT_gripper2base = np.column_stack((R_gb[i], T_gb[i]))
            RT_gripper2base = np.row_stack((RT_gripper2base, np.array([0, 0, 0, 1])))
            # print(RT_gripper2base)

            RT_camera_to_gripper = np.column_stack((R_cb, T_cb))
            RT_camera_to_gripper = np.row_stack((RT_camera_to_gripper, np.array([0, 0, 0, 1])))
            # print(RT_camera_to_gripper)#这个就是手眼矩阵

            RT_target_to_camera = np.column_stack((R_tc[i], T_tc[i]))
            RT_target_to_camera = np.row_stack((RT_target_to_camera, np.array([0, 0, 0, 1])))
            # print(RT_target_to_camera)
            RT_target_to_base = RT_gripper2base @ RT_camera_to_gripper @ RT_target_to_camera
            
            print("第{}次验证结果为:".format(i))
            print(RT_target_to_base)
            print('')

In [115]:
check_result(R_camera2gripper, T_camera2gripper, R_gripper2base_list, T_gripper2base_list, R_target2camera_list, T_target2camera_list)

第0次验证结果为:
[[-0.01722417 -0.99937133  0.03098491  0.68437866]
 [-0.99793363  0.01910149  0.06134142  1.90896355]
 [-0.06189521 -0.02986416 -0.99763574  0.73303305]
 [ 0.          0.          0.          1.        ]]

第1次验证结果为:
[[ 0.02066768 -0.99944474  0.02614028  0.67621376]
 [-0.99976306 -0.02048017  0.0074174   1.84671342]
 [-0.00687802 -0.02628697 -0.99963062  0.71416245]
 [ 0.          0.          0.          1.        ]]

第2次验证结果为:
[[ 0.03123113 -0.99885416  0.03626569  0.67066878]
 [-0.99948606 -0.03094689  0.00835858  1.838007  ]
 [-0.00722592 -0.03650851 -0.99930731  0.70895779]
 [ 0.          0.          0.          1.        ]]

第3次验证结果为:
[[ 2.74605888e-02 -9.99188442e-01  2.94601882e-02  6.67058648e-01]
 [-9.99622567e-01 -2.74098490e-02  2.12524727e-03  1.83709816e+00]
 [-1.31572547e-03 -2.95067411e-02 -9.99563612e-01  7.10423453e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

第4次验证结果为:
[[ 2.61993549e-02 -9.99525803e-01  1.61685206e-02  6.61745891e

In [111]:
def write_matrix_to_txt(file_path, matrix):
    with open(file_path, 'w') as f:
        np.savetxt(f, matrix.flatten()[np.newaxis,:])
def main():
    input_dir = "/Users/coulson/Downloads/23.7.7/7.7_rgb_one_circle/train/pose3"
    output_dir = "/Users/coulson/Downloads/23.7.7/7.7_rgb_one_circle/train/tsai_pose3"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if os.path.exists(input_dir + '/.DS_Store'):
        os.remove(input_dir + '/.DS_Store')
        
    T = RT_camera_to_gripper

    files = sorted(os.listdir(input_dir))
    
    for i in range(len(files)):
        current_absolute_matrix = read_matrix_from_txt(os.path.join(input_dir, files[i])) @ T
        write_matrix_to_txt(os.path.join(output_dir, files[i]), current_absolute_matrix)

if __name__ == "__main__":
    main()



### ros csv文件制作

In [ ]:
img_path = '/Users/coulson/Downloads/hand_eye/checkboard/select'
gt_path = '/Users/coulson/Downloads/hand_eye/pose_1.txt'
qvec, t = [], [] 
c2w_mats = []
# gt
file= open(gt_path)
f = file.read()
gt_msg = np.genfromtxt(StringIO(f), dtype=float)
# img
img_file = os.listdir(img_path)
img_len = len(img_file)
gt_len = len(gt_msg)
if os.path.exists(img_path + '/.DS_Store'):
        os.remove(img_path + '/.DS_Store')
img_msg = sorted([np.float64(x[:-4]) for x in img_file])
print(img_len)
print(gt_len)

In [ ]:
##########################################
#!!!!!!特别需要注意输入四元数是[w x y z]!!!!!!
##########################################

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

def change(array):
    new_array = np.concatenate((array[-1:], array[:-1])) # [w x y z] -> [x y z w]
    return new_array

import math

def quaternion_to_euler(qw, qx, qy, qz):
    # roll (x-axis rotation)
    sinr_cosp = 2 * (qw * qx + qy * qz)
    cosr_cosp = 1 - 2 * (qx * qx + qy * qy)
    roll = math.atan2(sinr_cosp, cosr_cosp)

    # pitch (y-axis rotation)
    sinp = 2 * (qw * qy - qz * qx)
    if abs(sinp) >= 1:
        # use 90 degrees if out of range
        pitch = math.copysign(math.pi / 2, sinp)
    else:
        pitch = math.asin(sinp)

    # yaw (z-axis rotation)
    siny_cosp = 2 * (qw * qz + qx * qy)
    cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
    yaw = math.atan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw  # in radians

# testing with some quaternion values
RX, RY, RZ = quaternion_to_euler(qw, qx, qy, qz)
print(RX, RY, RZ)
# print(f"RX: {math.degrees(RX)}, RY: {math.degrees(RY)}, RZ: {math.degrees(RZ)}")  # convert to degrees



In [ ]:
data = []
qvec, t = [], [] 
for i in range(len(img_msg)):
    t_ = img_msg[i]
    for j in range(len(gt_msg)):
        l, r = gt_msg[j][0] * 10**9, gt_msg[j + 1][0] * 10**9 ## 这里要根据截取图片的时间戳进行更改
        if img_msg[i] >= l and img_msg[i] <= r:
            l, r = gt_msg[j][0] * 10**9, gt_msg[j + 1][0] * 10**9 ## 这里要根据截取图片的时间戳进行更改
            q_s, q_e = Quaternion(change(gt_msg[j][-4:])), Quaternion(change(gt_msg[j + 1][-4:]))
            # q_s, q_e = Quaternion(gt_msg[j][-4:]), Quaternion(gt_msg[j + 1][-4:])
            dt = (t_ - l)/(r - l) 
            qvec.append(Quaternion.slerp(q_s, q_e, amount=dt))
            trans = dt * gt_msg[j + 1][1:4] + (1 - dt) * gt_msg[j][1:4] # 这里存疑：考虑平移是线性的，所以直接线性插值了
            t.append(trans)
            break
c2w_mats = []
X = []
bottom = np.array([0,0,0,1.]).reshape([1,4])

for x,y in zip(qvec, t):
#     R = qvec2rotmat(x)  
#     assert(np.isclose(np.linalg.det(R), 1.))
# #     #-----309-----#
# # 这个换位操作就等价于c2w右乘一个旋转矩阵
# #     R = np.concatenate([R[:, 0:1], -R[:, 2:3], R[:, 1:2]], 1) 
# #     #-----309-----#
#     t1 = y.reshape([3,1])
#     m = np.concatenate([np.concatenate([R, t1], 1), bottom], 0)
#     c2w_mats.append(m)
# c2w_mats = np.stack(c2w_mats, 0)
# # c2w_mats = np.linalg.inv(c2w_mats) # 读进来的矩阵就是c2w
# c2w_mats.shape
    RX, RY, RZ = quaternion_to_euler(x[0], x[1], x[2], x[3])
    t1 = y.reshape([3])
    data.append(["hand", t1[0], t1[1], t1[2], math.degrees(RX), math.degrees(RY), math.degrees(RZ)])
    print(math.degrees(RX), math.degrees(RY), math.degrees(RZ))
    print(t1)

In [ ]:
import csv
# 创建新的csv文件并写入数据
with open("data.csv", "w", newline="") as file:
    writer = csv.writer(file)
    # 写入数据
    for row in data:
        writer.writerow(row)
        if row[0] == "hand":
            writer.writerow([])

In [ ]:
nmsl = []
import math
K = np.array([[612.9027021585601, 0, 320.2485309181694],
              [0, 613.046755187905, 245.6729974189155],
              [0, 0, 1]])
distortion = np.array([[0.1280414029722492, -0.2978959338068587, -0.0013245790194379178, 0.0003422764625357527]]).reshape([4, 1])
# distortion = np.zeros(4).reshape([4, 1])
cnt = 0
image_directory = '/Users/coulson/Downloads/hand_eye/checkboard/select'  # directory containing images
output_directory = '/Users/coulson/Downloads/hand_eye/checkboard/cali4'

rvec_list = []
tvec_list = []
for filename in sorted(os.listdir(image_directory)):
    if filename.endswith('.png'):  # add other file types if needed
        img_path = os.path.join(image_directory, filename)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (11, 8), None)
        # print(len(corners))
#         # 提高精度
        term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 100, 0.001) # 指定亚像素计算迭代标注 
        cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), term) # 亚像素检测，更新corners

        corner_points = np.zeros((2, corners.shape[0]), dtype=np.float64)

        for i in range(corners.shape[0]):
            corner_points[:, i] = corners[i, 0, :]
        object_points = np.zeros((3, 11 * 8), dtype=np.float64) # 为棋盘格的每一个内角点生成对应的世界坐标
#         count = 0
#         for i in range(8):
#             for j in range(11):
#                 object_points[:2, count] = np.array(
#                     [(11 - j - 1) * 0.03, # 输出矩阵的单位与这里保持一致，选择 m 作为单位
#                      (8 - i - 1) * 0.03])
#                 count += 1
        count = 0
        for i in range(8):  # 8 rows
            for j in range(11):  # 11 columns
                object_points[:2, count] = np.array(
                    [j * 0.03, # 选择 m 作为单位
                     i * 0.03])
                count += 1
            
        retval, rvec, tvec = cv2.solvePnP(object_points.T, corner_points.T, K, distCoeffs=distortion)
#         print(cnt)
#         print([math.degrees(rvec[0]), math.degrees(rvec[1]), math.degrees(rvec[2])])
#         print(tvec.reshape(3))
        nmsl.append(["eye", tvec.reshape(3)[0], tvec.reshape(3)[1], tvec.reshape(3)[2], math.degrees(rvec[0]), math.degrees(rvec[1]), math.degrees(rvec[2]) ])
        Matrix_target2camera = np.column_stack(((cv2.Rodrigues(rvec))[0], tvec))
        Matrix_target2camera = np.row_stack((Matrix_target2camera, np.array([0, 0, 0, 1])))
        # Save to txt file
        txt_filename = os.path.join(output_directory, '{:04d}.txt'.format(cnt))
        np.savetxt(txt_filename, Matrix_target2camera.flatten()[np.newaxis,:], fmt="%f")
        cnt += 1   


In [ ]:
# 读取现有的csv文件
with open('data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
cnt = 0
# # # 插入新的数据
for i in range(1, len(nmsl) * 2, 2):
    data[i] = nmsl[cnt]
    cnt += 1
# # 写回csv文件
with open('data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)

### Verification

In [122]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R
from numpy.linalg import inv

pose_dir = '/Users/coulson/23.7.9/calibration/pose_cali'
pose_filenames = sorted(os.listdir(pose_dir))

# set data
# T_b_t can be any value. No effection on results
R_b_t = np.eye(3)
t_b_t = np.array([0, 0, 0])
T_b_t = np.vstack((np.hstack((R_b_t, t_b_t.reshape(-1,1))), [0,0,0,1]))

# give "T_c_g" (or T_g_c) to generate the T_b_g
theta_x = np.pi/5
theta_y = np.pi/6
theta_z = np.pi/9

rotX = np.array([[1, 0, 0],
                 [0, np.cos(theta_x), np.sin(theta_x)],
                 [0, -np.sin(theta_x), np.cos(theta_x)]])
rotY = np.array([[np.cos(theta_y), 0, -np.sin(theta_y)],
                 [0, 1, 0],
                 [np.sin(theta_y), 0, np.cos(theta_y)]])
rotZ = np.array([[np.cos(theta_z), np.sin(theta_z), 0],
                 [-np.sin(theta_z), np.cos(theta_z), 0],
                 [0, 0, 1]])

R_c_g =  rotZ @ rotY @ rotX
t_c_g = np.array([1, 2, 3])
T_c_g = np.vstack((np.hstack((R_c_g, t_c_g.reshape(-1,1))), [0,0,0,1]))
T_g_c = inv(T_c_g)

# generate grigper2base
N = 10
R_gripper2base_list = []
T_gripper2base_list = []
R_target2camera_list = []
T_target2camera_list = []

for i in range(N):

    T_t_c = read_matrix_from_txt(os.path.join(pose_dir, pose_filenames[i]))
    
    T_b_g = T_b_t @ T_t_c @ T_c_g  # generate T_b_g based on chain mul.
    
    R_gripper2base_list.append(T_b_g[:3, :3])
    T_gripper2base_list.append(T_b_g[:3, 3])
    R_target2camera_list.append(T_t_c[:3, :3])
    T_target2camera_list.append(T_t_c[:3, 3])

# X = tsai(Gij_list, Cij_list)  # This function needs to be implemented in Python
R_camera2gripper, T_camera2gripper = cv2.calibrateHandEye(R_gripper2base_list, T_gripper2base_list,
                                                            R_target2camera_list, T_target2camera_list, method = cv2.CALIB_HAND_EYE_PARK)

X = np.column_stack((R_camera2gripper, T_camera2gripper))
X = np.row_stack((X, np.array([0, 0, 0, 1])))
print(X)
print(T_g_c)

Error = np.linalg.norm(X - T_g_c)
print(Error)


[[ 0.86334263 -0.39340118 -0.31603008 -1.52682482]
 [ 0.49264635  0.79270409  0.35905404 -0.10496351]
 [-0.10926605  0.46567772 -0.87818289 -3.69105759]
 [ 0.          0.          0.          1.        ]]
[[ 0.81379768 -0.29619813  0.5        -1.72140142]
 [ 0.55286884  0.6597101  -0.50903696 -0.34517816]
 [-0.17907925  0.69068752  0.70062927 -3.30418359]
 [ 0.          0.          0.          1.        ]]
2.0606043051604632
